In [1]:
from matplotlib import pyplot as plt
from astropy.io import fits
import numpy as np
from scipy import ndimage
import re
import glob

from astropy.utils.data import get_file_contents
import astropy.units as u

%matplotlib inline
from IPython.utils import io

/home/panoptes/anaconda3/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
# Load the PANOPTES module dir
import sys
sys.path.append('../..')

# For development
%load_ext autoreload
%autoreload 2

from panoptes.utils.images import *

INFO:PanoptesLogger:

INFO:PanoptesLogger:********************************************************************************
INFO:PanoptesLogger:

DEBUG:PanoptesLogger:Adding warning logging to: PanError
DEBUG:PanoptesLogger:Adding warning logging to: SerialData
DEBUG:PanoptesLogger:Adding warning logging to: PanStateMachine
DEBUG:PanoptesLogger:Adding warning logging to: AbstractCamera
DEBUG:PanoptesLogger:Adding warning logging to: Observation
DEBUG:PanoptesLogger:Adding warning logging to: Target
DEBUG:PanoptesLogger:Adding warning logging to: Scheduler
DEBUG:PanoptesLogger:Adding warning logging to: Observatory
DEBUG:PanoptesLogger:Adding warning logging to: Panoptes
DEBUG:PanoptesLogger:Adding warning logging to: Messaging
DEBUG:PanoptesLogger:Adding warning logging to: EnvironmentalMonitor
DEBUG:PanoptesLogger:Adding warning logging to: Webcams
DEBUG:PanoptesLogger:Adding warning logging to: PanSensors


In [4]:
cd $d

/media/panoptes/panoptesdataMLO/images/20150920/raw


In [34]:
cr2_files = glob.glob('{}/01_1*.cr2'.format(d))
cr2_files.sort()
cr2_files

['/var/panoptes/images/20150920/raw/01_11.cr2',
 '/var/panoptes/images/20150920/raw/01_12.cr2',
 '/var/panoptes/images/20150920/raw/01_120.cr2',
 '/var/panoptes/images/20150920/raw/01_121.cr2',
 '/var/panoptes/images/20150920/raw/01_122.cr2',
 '/var/panoptes/images/20150920/raw/01_123.cr2',
 '/var/panoptes/images/20150920/raw/01_124.cr2',
 '/var/panoptes/images/20150920/raw/01_125.cr2',
 '/var/panoptes/images/20150920/raw/01_126.cr2',
 '/var/panoptes/images/20150920/raw/01_127.cr2',
 '/var/panoptes/images/20150920/raw/01_128.cr2',
 '/var/panoptes/images/20150920/raw/01_129.cr2',
 '/var/panoptes/images/20150920/raw/01_140.cr2',
 '/var/panoptes/images/20150920/raw/01_141.cr2',
 '/var/panoptes/images/20150920/raw/01_142.cr2',
 '/var/panoptes/images/20150920/raw/01_143.cr2',
 '/var/panoptes/images/20150920/raw/01_144.cr2',
 '/var/panoptes/images/20150920/raw/01_145.cr2',
 '/var/panoptes/images/20150920/raw/01_146.cr2',
 '/var/panoptes/images/20150920/raw/01_150.cr2',
 '/var/panoptes/images

In [35]:
pixel_scale = ((206.265) * (18.5 * u.micron) / (85 * u.mm)).cgs * u.arcsecond

In [44]:
%pdb

Automatic pdb calling has been turned ON


In [47]:
def compare(f0, f1, pixel_scale=0.):
    raw_data_00 = read_pgm(cr2_to_pgm(f0))
    raw_data_01 = read_pgm(cr2_to_pgm(f1))

    
    x0, y0 = measure_offset(raw_data_00,raw_data_01, box_width=250)
    
    #with io.capture_output() as captured:    
    #    !/var/panoptes/bin/measure_offset $f0 $f1 
        
    #x1 = float(get_file_contents('xcent.txt'))
    #y1 = float(get_file_contents('ycent.txt'))

    #return ( x0, y0, x1, y1 )
    return (x0, y1)

In [ ]:
comparison = {}

for i in np.arange(len(cr2_files)):
    if i < len(cr2_files) - 1:
        print(i)
        comparison[i] = compare(cr2_files[i], cr2_files[i+1], pixel_scale)
        
comparison

0


IndexError: too many indices for array

> /var/panoptes/POCS/panoptes/utils/images.py(110)measure_offset()
    109 
--> 110     y_mean = same_target[:,1].mean()
    111     x_mean = same_target[:,0].mean()

ipdb> print(same_target)
[]


In [ ]:
x0 = [v[0] for v in comparison.values()]
y0 = [v[1] for v in comparison.values()]
#x1 = [v[2] for v in comparison.values()]
#y1 = [v[3] for v in comparison.values()]

r = np.arange(len(comparison))

plt.scatter(r, x0, color='b', linewidths=2., label="X Difference")
plt.scatter(r, y0, color='r', linewidths=2., label="Y Difference")
#plt.scatter(r, x1, color='g', linewidths=2.)
#plt.scatter(r, y1, color='k', linewidths=2.)
plt.legend()

In [32]:
x1 * pixel_scale.to(u.ms)

UnitConversionError: 'arcsec' (angle) and 'ms' (time) are not convertible